# BertViz Interactive Demo: Head View
## **Scroll down for pre-loaded visualizations** 👇👇👇👇👇👇

In [1]:
from bertviz import *
from bertviz.neuron_view import show
from transformers import GPT2Tokenizer, GPT2LMHeadModel,GPT2Model
import torch

/opt/conda/envs/hug42/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Retreive attention weights

model_dir = "/mnt/petrelfs/guoyiqiu/coding/huggingface_models/"
model_dir = ""
model_version = 'gpt2'
model = GPT2LMHeadModel.from_pretrained(model_dir+model_version, output_attentions=True)
tokenizer = GPT2Tokenizer.from_pretrained(model_dir+model_version)

In [8]:
import torch
import os 
import json

from transformers import GPT2Tokenizer, GPT2LMHeadModel,GPT2Config

# Initialize tokenizer and model
model_dir = "/mnt/petrelfs/guoyiqiu/coding/huggingface_models/gpt2-math/small"
with open(os.path.join(model_dir,"model_config.json"), "r") as f:
    config_dict = json.load(f)
tokenizer = GPT2Tokenizer.from_pretrained(model_dir)
model = GPT2LMHeadModel(GPT2Config.from_dict(config_dict))
model.load_state_dict(torch.load(os.path.join(model_dir,"saved_models/grok_1678074595/10000.pth"),
                                 map_location=torch.device('cpu'))['model'])

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/petrelfs/guoyiqiu/coding/huggingface_models/gpt2-math/small/model_config.json'

In [ ]:
from gpt2_math.train_gpt import *

config = Config()
Config.data_dir='gpt2_math/data'
train,test = gen_train_test(config = config, tokenizer = tokenizer)
train_acc, wrong = eval_model(config = config, model = model, tokenizer = tokenizer, data = train)

In [8]:
res = model.generate(input_ids = torch.tensor([tokenizer.encode("17+47=")]), max_new_tokens = 2)
print(tokenizer.decode(res[0]))

17+47=64


In [5]:
text = "The quick brown fox jumps over the lazy dogs."
text = "Context: Human die. Tom is human. judugement: Tom would die. Is the judgement correct? The answer is yes"
text = "Mr and Mrs Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much. They were the last people you'd expect to be involved in anything strange or mysterious, because they just didn't hold with such nonsense. Mr Dursley was the director of a firm called Grunnings, which made drills."
text = "13+49=62"
inputs = tokenizer.encode_plus(text, return_tensors='pt', add_special_tokens=True)
input_ids = inputs['input_ids']
res = model(input_ids,output_attentions=True)
xo = []
for i in range(len(res['past_key_values'])):
    v = res['past_key_values'][i][1] # [bsz, num_heads, num_tokens, head_dim]
    if 'transformer' in dir(model):
        attn = model.transformer.h[i].attn
    else:
        attn = model.h[i].attn
    num_heads = v.shape[1]
    attn_head_size = v.shape[-1]
    v = attn._merge_heads(v,num_heads,attn_head_size)
    v = attn.c_proj(v)
    v = attn._split_heads(v,num_heads,attn_head_size)
    v = torch.norm(v,dim=-1) # [bsz, num_heads, num_tokens] value vector norm
    v = v.repeat(1,1,v.shape[-1]).reshape(v.shape[0],v.shape[1],-1,v.shape[-1]) # [bsz, num_heads, num_tokens, num_tokens]
    xo.append(v)
attention = []
for x, a in zip(xo, res['attentions']):
    causal_mask = torch.triu(torch.ones(a.shape[-1],a.shape[-1]),diagonal=1)*-1e9
    attention.append(torch.softmax(a * x + causal_mask, dim=-1))
# attention = res['attentions']
input_id_list = input_ids[0].tolist() # Batch index 0
tokens = tokenizer.convert_ids_to_tokens(input_id_list)

In [6]:
# from IPython.core.display import HTML
# display(HTML('<script src="/static/components/requirejs/require.js"></script>'))
# Above two lines only needed when running in Colab
head_view(attention, tokens)

<IPython.core.display.Javascript object>

In [7]:
# from IPython.core.display import HTML
# display(HTML('<script src="/static/components/requirejs/require.js"></script>'))
# Above two lines only needed when running in Colab
model_view(attention, tokens)

<IPython.core.display.Javascript object>